# 

In [17]:
import pandas as pd
import numpy as np
import pickle as pkl
import peptides
import propy

import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold

In [2]:
# gets training and testing data, as used by Witten & Witten (2019). 
data_dir = "models/Saved_variables/"
ecoli_train_with_c = pd.read_pickle(f'{data_dir}ecoli_train_with_c_df.pkl')
ecoli_train_no_c = pd.read_pickle(f'{data_dir}ecoli_train_no_c_df.pkl')

ecoli_test = pd.read_pickle(f'{data_dir}ecoli_test_df.pkl')

ecoli_df = pd.read_pickle(f'{data_dir}ecoli_all_df.pkl')
ecoli_df_no_c = pd.read_pickle(f'{data_dir}ecoli_all_no_c_df.pkl')

all_df = pd.read_pickle(f'{data_dir}all_df.pkl')

ecoli_train_train_no_c = pd.read_pickle(f'{data_dir}ecoli_train_train_no_c_df.pkl')
ecoli_validate_no_c = pd.read_pickle(f'{data_dir}ecoli_validate_no_c_df.pkl')

In [3]:

ecoli_train_no_c.head(3)

,level_0,index,sequence,bacterium,value,is_modified,has_unusual_modification,has_cterminal_amidation,_datasource_has_modifications,_sequence_has_modifications,modification_verified
0,0,0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,E. coli,2.09995,0.0,0.0,0.0,1.0,1.0,1.0
1,1,1,AAAAAAAIKMLMDLVNERIMALNKKAKK,E. coli,1.00000,1.0,0.0,1.0,1.0,1.0,1.0
2,2,2,AAAKAALNAVLVGANA,E. coli,1.90309,0.0,0.0,0.0,1.0,1.0,1.0


In [4]:
print(
    f"{len(ecoli_train_with_c)=}",
    f"{len(ecoli_test)=}"
)

len(ecoli_train_with_c)=4050 len(ecoli_test)=509


In [5]:
titles=['Aliphatic Index', 'Boman Index', 'Charge pH 3', 'Charge pH 7', 'Charge pH 11', 'Hydrophobicity', 'Instability Index', 'Isoelectric Point', 'Molecular Weight']

In [8]:
class SVRonPhysicoChemicalProps():
    def __init__(self, svr_C=1.0, svr_epsilon=0.1):
        
        self.svr_C = svr_C
        self.svr_epsilon = svr_epsilon

        self.scaler = StandardScaler()
        self.svr = SVR(C=svr_C, epsilon=svr_epsilon)

    def get_physico_chemical_props(self,sequences:list[str]|str):

        if isinstance(sequences, str):
            sequences = [sequences]

        titles=[
            'Aliphatic Index', 'Boman Index', 
            'Charge pH 3', 'Charge pH 7', 'Charge pH 11', 
            'Hydrophobicity', 'Instability Index', 
            'Isoelectric Point', 'Molecular Weight'
        ]
        physico_props = {
            "sequence":[]
        }
        for title_ in titles:
            physico_props[title_] = []
            
        for seq_ in sequences:
            peptide_ = peptides.Peptide(seq_)
            physico_props["sequence"].append(seq_)
            for title_ in titles:
                if title_ == "Aliphatic Index":
                    physico_props[title_].append(peptide_.aliphatic_index())
                elif title_ == "Boman Index":
                    physico_props[title_].append(peptide_.boman())
                elif title_ == "Charge pH 3":
                    physico_props[title_].append(peptide_.charge(pH=3))
                elif title_ == 'Charge pH 7':
                    physico_props[title_].append(peptide_.charge(pH=7))
                elif title_ == "Charge pH 11":
                    physico_props[title_].append(peptide_.charge(11))
                elif title_ == "Hydrophobicity":
                    physico_props[title_].append(peptide_.hydrophobicity())
                elif title_ == "Instability Index":
                    physico_props[title_].append(peptide_.instability_index())
                elif title_ == "Isoelectric Point":
                    physico_props[title_].append(peptide_.isoelectric_point())
                elif title_ == "Molecular Weight":
                    physico_props[title_].append(peptide_.molecular_weight())
        
        output = pd.DataFrame(data=physico_props)
        return output[ output.columns[1:] ]

    def fit(self, X, y, input_type="sequences"):
        """
        Arguments:
        ----------
            X: array, input values
            y: array, target values
            input_type: str, choices=["sequences","physicochemical_properties"]
        """
        if input_type not in ["sequences", "physicochemical_properties"]:
            raise ValueError("input_type must be one of ['sequences','physicochemical_properties']")
        
        if input_type=="sequences":
            physicochemical_props_ = self.get_physico_chemical_props(X)
        else:
            physicochemical_props_ = X

        # standardize data
        self.scaler.fit(physicochemical_props_)
        physicochemical_props_ = self.scaler.transform(physicochemical_props_)

        # fit svr model
        self.svr.fit(physicochemical_props_, y)

    def predict(self, sequences:list[str]|str):
        """
        wrapper predict method to automatically get physicochemical properties given (a) new sequence(s)

        Arguments:
        ----------
        sequences: str or list[str], sequences to predict off of

        Returns:
        --------
        predicted values, 
        """
        if isinstance(sequences, str):
            sequences = [sequences]
        
        physicochemical_props_ = self.get_physico_chemical_props(sequences)

        # put through scaler
        physicochemical_props_ = self.scaler.transform(physicochemical_props_)
        
        y_predicted = self.svr.predict(physicochemical_props_)
        return y_predicted

In [9]:
# train_X = ecoli_train_physico_props[ecoli_train_physico_props.columns[1:]] # ignore col=='sequence'
train_X = ecoli_train_no_c.sequence
train_Y = ecoli_train_no_c.value

# test_X = ecoli_test_physico_props[ecoli_test_physico_props.columns[1:]] # ignore col=='sequence'
test_X = ecoli_validate_no_c.sequence
test_Y = ecoli_validate_no_c.value

In [10]:
# train_X = ecoli_train_physico_props[ecoli_train_physico_props.columns[1:]] # ignore col=='sequence'
train_X = ecoli_train_with_c.sequence
train_Y = ecoli_train_with_c.value

# test_X = ecoli_test_physico_props[ecoli_test_physico_props.columns[1:]] # ignore col=='sequence'
test_X = ecoli_test.sequence
test_Y = ecoli_test.value

In [11]:
C_values = [0.01,0.1,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,10,15,20,25,50,75,100,1e3,1e4]

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(train_X, 
                                                      train_Y, 
                                                      test_size=0.2,
                                                      random_state=42, 
                                                      shuffle=True
)

print(f"{len(X_train)=} | {len(X_valid)=} ")

len(X_train)=3240 | len(X_valid)=810 


In [13]:
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

In [14]:
results = {}

for i, c_ in enumerate(C_values):
    avg_test_mse  = 0
    avg_train_mse = 0
    print(f"{i=}")
    for ii, (train_index, test_index) in enumerate(kf.split(train_X)):
        x_train_ = train_X.iloc[train_index]
        y_train_ = train_Y.iloc[train_index]
        
        x_test_  = train_X.iloc[ test_index]
        y_test_  = train_Y.iloc[ test_index]
        
        svr = SVRonPhysicoChemicalProps(svr_C=c_)
        
        svr.fit(x_train_, y_train_)
        train_mse_ = mean_squared_error(y_train_, svr.predict(x_train_))
        test_mse_  = mean_squared_error( y_test_, svr.predict( x_test_))

        avg_train_mse += train_mse_ 
        avg_test_mse  +=  test_mse_
        
    avg_train_mse = avg_train_mse/N_SPLITS
    avg_test_mse  =  avg_test_mse/N_SPLITS
    
    print(f"{c_=}, {avg_train_mse=}, {avg_test_mse=}")

i=0
c_=0.01, avg_train_mse=0.47473864605873795, avg_test_mse=0.47962084528075033
i=1
c_=0.1, avg_train_mse=0.4228978214577788, avg_test_mse=0.4392925066313168
i=2
c_=0.5, avg_train_mse=0.380225683644596, avg_test_mse=0.4131430627831881
i=3
c_=1.0, avg_train_mse=0.36326752502857784, avg_test_mse=0.40703667016486894
i=4
c_=1.5, avg_train_mse=0.353757836591585, avg_test_mse=0.40557255078051513
i=5
c_=2.0, avg_train_mse=0.34750089007764623, avg_test_mse=0.4047257916483623
i=6
c_=2.5, avg_train_mse=0.34278723173329717, avg_test_mse=0.40432221314044836
i=7
c_=3.0, avg_train_mse=0.33896086636570233, avg_test_mse=0.4045545216306802
i=8
c_=3.5, avg_train_mse=0.3358360350349702, avg_test_mse=0.40515221352023156
i=9
c_=4.0, avg_train_mse=0.33313083172784375, avg_test_mse=0.40539743516173116
i=10
c_=4.5, avg_train_mse=0.33071674056094447, avg_test_mse=0.4058710053590434
i=11
c_=5.0, avg_train_mse=0.3284290537204938, avg_test_mse=0.4059588405365241
i=12
c_=10, avg_train_mse=0.3133380469993349, avg_

In [41]:
svr.fit(train_X, train_Y)

In [42]:
svr.predict("GKVWDWIKSAAKKIWSSEPVSQLKGQVLNAAKNYVAEKIGATPT")

array([0.57234479])

In [52]:
svr.predict(test_X)[0:5]

array([0.18111109, 0.5156799 , 0.40233027, 1.12336439, 0.13583254])

In [44]:
train_X.shape

(4050,)

In [45]:
ecoli_test.shape

(509, 10)

In [46]:
test_X.shape

(509,)

In [47]:
train_mse = mean_squared_error(train_Y, svr.predict(train_X))
test_mse  = mean_squared_error( test_Y, svr.predict( test_X))
print(f"{train_mse=}, {test_mse=}")

train_mse=0.36311391112632696, test_mse=0.3716201825484443


### what are some baseline errors?

always predicting the average

In [58]:
class PredictMean():
    def __init__(self):
        self.mean = None
        
    def fit(self, X, y, input_type="sequences"):

        self.mean = y.mean()

    def predict(self, X):
        return self.mean*np.ones_like(X)

In [59]:
mean_predictor = PredictMean()
mean_predictor.fit(train_X, train_Y)
print(
    mean_squared_error(train_Y, mean_predictor.predict(train_X)),
    mean_squared_error( test_Y, mean_predictor.predict( test_X))
)

0.5984786684010958 0.6123846394238136


multi-variable linear regression

In [53]:
class LinearRegPhysico():
    def __init__(self):
        
        self.scaler = StandardScaler()
        self.reg = LinearRegression()

    def get_physico_chemical_props(self,sequences:list[str]|str):

        if isinstance(sequences, str):
            sequences = [sequences]

        titles=[
            'Aliphatic Index', 'Boman Index', 
            'Charge pH 3', 'Charge pH 7', 'Charge pH 11', 
            'Hydrophobicity', 'Instability Index', 
            'Isoelectric Point', 'Molecular Weight'
        ]
        physico_props = {
            "sequence":[]
        }
        for title_ in titles:
            physico_props[title_] = []
            
        for seq_ in sequences:
            peptide_ = peptides.Peptide(seq_)
            physico_props["sequence"].append(seq_)
            for title_ in titles:
                if title_ == "Aliphatic Index":
                    physico_props[title_].append(peptide_.aliphatic_index())
                elif title_ == "Boman Index":
                    physico_props[title_].append(peptide_.boman())
                elif title_ == "Charge pH 3":
                    physico_props[title_].append(peptide_.charge(pH=3))
                elif title_ == 'Charge pH 7':
                    physico_props[title_].append(peptide_.charge(pH=7))
                elif title_ == "Charge pH 11":
                    physico_props[title_].append(peptide_.charge(11))
                elif title_ == "Hydrophobicity":
                    physico_props[title_].append(peptide_.hydrophobicity())
                elif title_ == "Instability Index":
                    physico_props[title_].append(peptide_.instability_index())
                elif title_ == "Isoelectric Point":
                    physico_props[title_].append(peptide_.isoelectric_point())
                elif title_ == "Molecular Weight":
                    physico_props[title_].append(peptide_.molecular_weight())
        
        output = pd.DataFrame(data=physico_props)
        return output[ output.columns[1:] ]

    def fit(self, X, y, input_type="sequences"):
        """
        Arguments:
        ----------
            X: array, input values
            y: array, target values
            input_type: str, choices=["sequences","physicochemical_properties"]
        """
        if input_type not in ["sequences", "physicochemical_properties"]:
            raise ValueError("input_type must be one of ['sequences','physicochemical_properties']")
        
        if input_type=="sequences":
            physicochemical_props_ = self.get_physico_chemical_props(X)
        else:
            physicochemical_props_ = X

        # standardize data
        self.scaler.fit(physicochemical_props_)
        physicochemical_props_ = self.scaler.transform(physicochemical_props_)

        # fit linear regression model
        self.reg.fit(physicochemical_props_, y)

    def predict(self, sequences:list[str]|str):
        """
        wrapper predict method to automatically get physicochemical properties given (a) new sequence(s)

        Arguments:
        ----------
        sequences: str or list[str], sequences to predict off of

        Returns:
        --------
        predicted values, 
        """
        if isinstance(sequences, str):
            sequences = [sequences]
        
        physicochemical_props_ = self.get_physico_chemical_props(sequences)

        # put through scaler
        physicochemical_props_ = self.scaler.transform(physicochemical_props_)
        
        y_predicted = self.reg.predict(physicochemical_props_)
        return y_predicted

In [54]:
linear_regression = LinearRegPhysico()

In [55]:
linear_regression.fit(train_X, train_Y)

In [61]:
print('using SVR..')
print(
    mean_squared_error(train_Y, svr.predict(train_X)),
    mean_squared_error( test_Y, svr.predict( test_X))
)

using SVR..
0.36311391112632696 0.3716201825484443


In [56]:
train_mse = mean_squared_error(train_Y, linear_regression.predict(train_X))
test_mse  = mean_squared_error( test_Y, linear_regression.predict( test_X))
print("using linear regression..")
print(f"{train_mse=}, {test_mse=}")

using linear regression..
train_mse=0.46687513744798514, test_mse=0.44147638488603846


In [60]:
print("using mean...")
print(
    mean_squared_error(train_Y, mean_predictor.predict(train_X)),
    mean_squared_error( test_Y, mean_predictor.predict( test_X))
)

using mean...
0.5984786684010958 0.6123846394238136
